In [1]:
import ee
#ee.Authenticate()

Enter verification code: 4/1Adeu5BXdcP98YmNI-WhjDS4jF_UHNC_RtqGPt9qudId5CVg7sd0DzKY4ZCA

Successfully saved authorization token.


In [4]:
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob
import geemap